### AM4 GFDL TSTORMS tracker output files inspection

In [1]:
import os 
import numpy as np 
from typing import Tuple

BASEDIR = "../data/am4aqua"
R = 1769.72


def haversine(lat_0:float,lat_1:float,lon_0:float,lon_1:float)->float:
    return 2*np.arcsin(np.sqrt(np.sin((lat_1 - lat_0)/2)**2 + np.cos(lat_1)*np.cos(lat_0)*np.sin((lon_1 - lon_0)/2)**2 ))

def bearing(lat_0:float,lat_1:float,lon_0:float,lon_1)->float:
    x = np.cos(lat_1)*np.sin(lon_1 - lon_0)
    y = np.cos(lat_0)*np.sin(lat_1) - np.sin(lat_0)*np.cos(lat_1)*np.cos(lon_1 - lon_0)
    return np.arctan2(x,y)


def get_mean_speed(lat_0:float,lat_1:float,lon_0:float,lon_1:float,dt:float=6.0)->Tuple[float,float]:
    """
    u,v
    """
    d = haversine(np.deg2rad(lat_0),np.deg2rad(lat_1),np.deg2rad(lon_0),np.deg2rad(lon_1))
    theta = bearing(np.deg2rad(lat_0),np.deg2rad(lat_1),np.deg2rad(lon_0),np.deg2rad(lon_1))
    
    return R*d*np.sin(theta)/dt,R*d*np.cos(theta)/dt

def coriolis_parameter(lat:float):
    """
    Coriolis parameter important TC parameter, proportional to earth roation rate 
    """
    return 2*np.sin(np.deg2rad(lat)) #

def parse_trav_file(path:str,sstmax:float):
    records = {}
    cur_idx = None
    with open(path) as file:
        for line in file:
            line = line.split()
            if line[0] == 'start':
                cur_idx = "_".join(line[1:])
                records[cur_idx] = []
            else:
                cur_data = list(map(float,line[:5]))
                cur_data.append(SST(cur_data[1],sstmax))
                cur_data.append(sstmax)
                cur_data.append(coriolis_parameter(cur_data[1]))
                if len(records[cur_idx]):
                    u,v = get_mean_speed(records[cur_idx][-1][1],cur_data[1],records[cur_idx][-1][0],cur_data[0])
                else:
                    u,v = np.nan,np.nan
                cur_data.append(u)
                cur_data.append(v)
            
                records[cur_idx].append(cur_data)
                ## start record end previous
    return records 
            
        
def SST(lat:float,sstmaxlat:float):
    if lat > 60 or lat < -60:
        return 0 
    elif  lat > sstmaxlat:
        return 27*(1 - np.sin(90*(lat*np.pi/180 - sstmaxlat*np.pi/180)/(60 - sstmaxlat))**2)
    else:
        return 27*(1 - np.sin(90*(lat*np.pi/180 - sstmaxlat*np.pi/180)/(60 + sstmaxlat))**2)




In [2]:
records = {}
for root,_,files in os.walk(BASEDIR):
    for file in files:
        if "trav" in file:
            sstmax = float(root[-2:])
            records[sstmax] = parse_trav_file(os.path.join(root,file),sstmax=sstmax)




In [3]:
import json
with open(os.path.join(BASEDIR,"am4_all_cyclones.json"),'w') as f:
    json.dump(records,f)